# Sentiment Analysis terhadap Vaksinisasi COVID-19 di Indonesia: Validation and Data Prediction
<h2>Tim Yaudahlah</h2>


---

Kaenova Mahendra Auditama<sup>1</sup><br>
Fendi Irfan Amorokhman<sup>2</sup><br>
Ananda Affan Fattahila<sup>3</sup><br>
<sup>1</sup><a href="mailto:kaenova@student.telkomuniversity.ac.id">kaenova@student.telkomuniversity.ac.id</a><br>
<sup>2</sup><a href="mailto:fendiirfan@student.telkomuniversity.ac.id">fendiirfan@student.telkomuniversity.ac.id</a><br>
<sup>3</sup><a href="mailto:affanfattahila@student.telkomuniversity.ac.id">affanfattahila@student.telkomuniversity.ac.id</a><br>
Informatics Engineering, Telkom University, Indonesia<br>
2021


# Performance Validation
Sebelum melakukan eksplorasi data menggunakan model yang telah kami buat, kami melakukan validasi performansi dari setiap model untuk melihat validitas dari setiap model.

## Installing Pre-requisite

In [4]:
!pip install tqdm transformers

     |████████████████████████████████| 2.6 MB 32.7 MB/s 
     |████████████████████████████████| 636 kB 47.1 MB/s 
     |████████████████████████████████| 895 kB 50.4 MB/s 
     |████████████████████████████████| 3.3 MB 44.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importing Modules and Library

In [1]:
# Connect to GEMASTIK XIV: Drive for modules
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/Shareddrives/GEMASTIK XIV: Yaudahlah/notebooks/modules')

In [5]:
from postprocessing import DataPostProcessing
from preprocessing import DataPreProcessing

import pandas as pd
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn
import statistics
from sklearn.utils import shuffle
from transformers import TFBertModel, TFBertForSequenceClassification
from keras.utils.np_utils import to_categorical

## Import Validation Data and Assigning Constant

In [5]:
VALIDATION_DATA_CSV_PATH = "/content/drive/Shareddrives/GEMASTIK XIV: Yaudahlah/data/validation/validation_test.csv" #@param {type:"string"}
df_val = pd.read_csv(VALIDATION_DATA_CSV_PATH)
df_val = shuffle(df_val)
df_val.reset_index(inplace=True, drop=True)

BERT_BASE_MODEL = 'indobenchmark/indobert-base-p2'#@param {type:"string"}
MAX_LENGTH = 128 #@param {type:"integer"}

#@markdown ---
#@markdown # Learning and Fitting Constant
LEARNING_RATE = 3e-6 #@param {type:"raw"}
EPSILON = 1e-08 #@param {type:"raw"}
CLIPNORM = 1.0 #@param {type:"raw"}

## Pre-processing Data

In [6]:
x_bert, y_bert, one_hot_mappings = DataPreProcessing.PreProcessBatchValidationBERT(df_val, BERT_BASE_MODEL, MAX_LENGTH)
x_lstm, y_lstm, one_hot_mappings = DataPreProcessing.PreProcessBatchValidationLSTM(df_val)
print('One Hot Mappings: ',one_hot_mappings)

DataPreProcessing[PreProcessBatch|OneHotEncodingLabels]: Below zero detected in labels data
DataPreProcessing[PreProcessBatch|OneHotEncodingLabels]: Non-Category datatype detected, converting to Category datatype
DataPreProcessing[PreProcessBatch|Tokenize]: Fetching Tokenizer



DataPreProcessing[PreProcessBatchValidation]: labels shape (80, 2), input shape (80, 2, 128)
DataPreProcessing[PreProcessBatch|OneHotEncodingLabels]: Below zero detected in labels data
DataPreProcessing[PreProcessBatch|OneHotEncodingLabels]: Non-Category datatype detected, converting to Category datatype
DataPreProcessing[PreProcessBatchValidation]: labels shape (80, 2), input shape (80,)
One Hot Mappings:  {0: -1, 1: 1}


In [7]:
df_val['labels'].value_counts()

 1    40
-1    40
Name: labels, dtype: int64

## Load Models

In [8]:
# Custom matrics
def get_f1(y_true, y_pred): #taken from old keras source code
  true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
  possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
  predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
  recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
  f1_val = 2*(precision*recall)/(precision+recall+tf.keras.backend.epsilon())
  return f1_val

### BERT Only

In [9]:
BERT_MODEL_H5_PATH = '/content/drive/Shareddrives/GEMASTIK XIV: Yaudahlah/models/@Final Model/BERT_P2_Balance_V2_D26M07Y2021 H08M35S48.h5' #@param {type:"string"}
model_bert = tf.keras.models.load_model(BERT_MODEL_H5_PATH, compile=False)
model_bert.compile(
      tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, epsilon=EPSILON, clipnorm=CLIPNORM), 
      loss = 'categorical_crossentropy', 
      metrics = [
        tf.keras.metrics.TruePositives(name='TruePositive'),
        tf.keras.metrics.FalsePositives(name='FalsePositive'),
        tf.keras.metrics.TrueNegatives(name='TrueNegative'),
        tf.keras.metrics.FalseNegatives(name='FalseNegative'), 
        tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
        get_f1
      ], 
      # loss_weights = None, 
      # sample_weight_mode = None, 
      # weighted_metrics = None, 
      # target_tensors = None
    )

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


### LSTM Only

In [10]:
LSTM_MODEL_H5_PATH = '/content/drive/Shareddrives/GEMASTIK XIV: Yaudahlah/models/@Final Model/LSTM_Balance_Final' #@param {type:"string"}
model_lstm = tf.keras.models.load_model(LSTM_MODEL_H5_PATH, compile=False)
model_lstm.compile(
      tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, epsilon=EPSILON, clipnorm=CLIPNORM), 
      loss = 'categorical_crossentropy', 
      metrics = [
        tf.keras.metrics.TruePositives(name='TruePositive'),
        tf.keras.metrics.FalsePositives(name='FalsePositive'),
        tf.keras.metrics.TrueNegatives(name='TrueNegative'),
        tf.keras.metrics.FalseNegatives(name='FalseNegative'), 
        tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
        get_f1
      ], 
      # loss_weights = None, 
      # sample_weight_mode = None, 
      # weighted_metrics = None, 
      # target_tensors = None
)

### BERT - LSTM

In [11]:
BERT_LSTM_MODEL_H5_PATH = '/content/drive/Shareddrives/GEMASTIK XIV: Yaudahlah/models/@Final Model/BERT_P2_LSTM_Balance_V2_D26M07Y2021 H11M00S17.h5' #@param {type:"string"}
model_bert_lstm = tf.keras.models.load_model(BERT_LSTM_MODEL_H5_PATH, compile=False)
model_bert_lstm.compile(
      tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, epsilon=EPSILON, clipnorm=CLIPNORM), 
      loss = 'categorical_crossentropy', 
      metrics = [
        tf.keras.metrics.TruePositives(name='TruePositive'),
        tf.keras.metrics.FalsePositives(name='FalsePositive'),
        tf.keras.metrics.TrueNegatives(name='TrueNegative'),
        tf.keras.metrics.FalseNegatives(name='FalseNegative'), 
        tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
        get_f1
      ], 
      # loss_weights = None, 
      # sample_weight_mode = None, 
      # weighted_metrics = None, 
      # target_tensors = None
    )

## Performance Validation

In [12]:
list_acc_f1 ={
    'bert':{
        'acc':[],
        'f1':[]
    },
    'lstm':{
        'acc':[],
        'f1':[]
    },
    'bert_lstm':{
        'acc':[],
        'f1':[]
    }
}

In [13]:
for i in range(10):
  print('============== SAMPLE {} =============='.format(i+1))
  print('Validation on BERT Model')
  val_bert= model_bert.evaluate([x_bert[:,0],x_bert[:,1]], y_bert)
  list_acc_f1['bert']['acc'].append(val_bert[5])
  list_acc_f1['bert']['f1'].append(val_bert[9])
  print('Validation on LSTM Model')
  val_lstm = model_lstm.evaluate(x_lstm, y_lstm) # x_string : list string
  list_acc_f1['lstm']['acc'].append(val_lstm[5])
  list_acc_f1['lstm']['f1'].append(val_lstm[9])
  print('Validation on BERT-LSTM Model')
  val_bert_lstm = model_bert_lstm.evaluate([x_bert[:,0],x_bert[:,1]], y_bert)
  list_acc_f1['bert_lstm']['acc'].append(val_bert_lstm[5])
  list_acc_f1['bert_lstm']['f1'].append(val_bert_lstm[9])

============== SAMPLE 1 ==============
Validation on BERT Model
3/3 [==============================] - 10s 216ms/step - loss: 0.7266 - TruePositive: 66.0000 - FalsePositive: 14.0000 - TrueNegative: 66.0000 - FalseNegative: 14.0000 - accuracy: 0.8250 - precision: 0.8250 - recall: 0.8250 - auc: 0.8767 - get_f1: 0.8125
Validation on LSTM Model
3/3 [==============================] - 29s 133ms/step - loss: 0.7292 - TruePositive: 56.0000 - FalsePositive: 24.0000 - TrueNegative: 56.0000 - FalseNegative: 24.0000 - accuracy: 0.7000 - precision: 0.7000 - recall: 0.7000 - auc: 0.7496 - get_f1: 0.6875
Validation on BERT-LSTM Model
3/3 [==============================] - 9s 218ms/step - loss: 0.5499 - TruePositive: 62.0000 - FalsePositive: 18.0000 - TrueNegative: 62.0000 - FalseNegative: 18.0000 - accuracy: 0.7750 - precision: 0.7750 - recall: 0.7750 - auc: 0.9029 - get_f1: 0.7708
============== SAMPLE 2 ==============
Validation on BERT Model
3/3 [==============================] - 1s 217ms/step - l

In [14]:
df_val = pd.DataFrame(list_acc_f1)

def avg(lst):
  sum = 0
  for i in range(len(lst)):
    sum += lst[i]
  return (sum / len(lst))


print("===== Model Performance =====")
for col in df_val:
  print('{:10s}    | acc: {:1.3f} with stdev {:1.3f}  |  f1: {:1.3f} with stdev {:1.3f}'.format(col, round(avg(df_val[col]['acc']), 3), round(statistics.stdev(df_val[col]['acc']), 3), round(avg(df_val[col]['f1']), 3), round(statistics.stdev(df_val[col]['f1']), 3)))


===== Model Performance =====
bert          | acc: 0.796 with stdev 0.022  |  f1: 0.786 with stdev 0.024
lstm          | acc: 0.700 with stdev 0.000  |  f1: 0.687 with stdev 0.000
bert_lstm     | acc: 0.794 with stdev 0.016  |  f1: 0.786 with stdev 0.016


## Create Prediction for Every CSV in a directory

In [16]:
import matplotlib
import os
matplotlib.style.use('ggplot')

IN_DIRECTORY = '/content/drive/Shareddrives/GEMASTIK XIV: Yaudahlah/data/validation/processed/without_replies_retweet' #@param {type:"string"}
OUT_DIRECTORY = '/content/drive/Shareddrives/GEMASTIK XIV: Yaudahlah/data/exploration' #@param {type:"string"}

if IN_DIRECTORY[len(IN_DIRECTORY)-1] != '/':
  IN_DIRECTORY = IN_DIRECTORY + '/'

if OUT_DIRECTORY[len(OUT_DIRECTORY)-1] != '/':
  OUT_DIRECTORY = OUT_DIRECTORY + '/'

files = os.listdir(IN_DIRECTORY)

one_hot_mappings = {
    0: -1,
    1: 1
}

for file in files:
    if file[len(file)-4:] == '.csv':
      print("=== Processing {} ===".format(file))
      df_temp = pd.read_csv(IN_DIRECTORY+file)
      x_bert = DataPreProcessing.PreProcessBatchWithoutLabelsBERT(df_temp, BERT_BASE_MODEL, MAX_LENGTH)
      x_lstm = DataPreProcessing.PreProcessBatchWithoutLabelsLSTM(df_temp)
      y_bert = model_bert.predict([x_bert[:,0],x_bert[:,1]])
      y_lstm = model_lstm.predict(x_lstm)
      y_bert_lstm = model_bert_lstm.predict([x_bert[:,0],x_bert[:,1]])
      labels_bert = DataPostProcessing.PostProcessBatchToList(y_bert, one_hot_mappings)
      labels_lstm = DataPostProcessing.PostProcessBatchToList(y_lstm, one_hot_mappings)
      labels_bert_lstm = DataPostProcessing.PostProcessBatchToList(y_bert_lstm, one_hot_mappings)
      df_temp['bert_predict'] = labels_bert
      df_temp['lstm_predict'] = labels_lstm
      df_temp['bert_lstm_predict'] = labels_bert_lstm
      df_temp.to_csv(OUT_DIRECTORY+file)
      print(f'\n')

=== Processing astrazenaca.csv ===
DataPreProcessing[PreProcessBatch|Tokenize]: Fetching Tokenizer


100%|██████████| 1934/1934 [00:00<00:00, 133520.71it/s]

DataPostProcessing[PostProcessBatch]: Post Processing Batch prediction with Labels {0: -1, 1: 1}
DataPostProcessing[PostProcessBatch]: Post Processing Batch prediction with Labels {0: -1, 1: 1}
DataPostProcessing[PostProcessBatch]: Post Processing Batch prediction with Labels {0: -1, 1: 1}




=== Processing berhasil.csv ===
DataPreProcessing[PreProcessBatch|Tokenize]: Fetching Tokenizer


100%|██████████| 4868/4868 [00:00<00:00, 180097.84it/s]

...

100%|██████████| 4074/4074 [00:00<00:00, 195431.97it/s]

DataPostProcessing[PostProcessBatch]: Post Processing Batch prediction with Labels {0: -1, 1: 1}
DataPostProcessing[PostProcessBatch]: Post Processing Batch prediction with Labels {0: -1, 1: 1}
DataPostProcessing[PostProcessBatch]: Post Processing Batch prediction with Labels {0: -1, 1: 1}




=== Processing sinopharm.csv ===
DataPreProcessing[PreProcessBatch|Tokenize]: Fetching Tokenizer


100%|██████████| 383/383 [00:00<00:00, 159208.96it/s]

DataPostProcessing[PostProcessBatch]: Post Processing Batch prediction with Labels {0: -1, 1: 1}
DataPostProcessing[PostProcessBatch]: Post Processing Batch prediction with Labels {0: -1, 1: 1}
DataPostProcessing[PostProcessBatch]: Post Processing Batch prediction with Labels {0: -1, 1: 1}
